In [ ]:
##### import modules #####

from os.path import join as opj
from nipype.interfaces.ants import ApplyTransforms
from nipype.interfaces.utility import IdentityInterface
from nipype.interfaces.freesurfer import FSCommand, MRIConvert
from nipype.interfaces.io import SelectFiles, DataSink, FreeSurferSource
from nipype.pipeline.engine import Workflow, Node, MapNode
from nipype.algorithms.misc import Gunzip 

In [ ]:
# FreeSurfer - Specify the location of the freesurfer folder
fs_dir = '/media/lmn/86A406A0A406933B2/TNAC_BIDS/derivatives/mindboggle/freesurfer_subjects/'
FSCommand.set_default_subjects_dir(fs_dir)

In [ ]:
##### set paths and define parameters #####

experiment_dir = '/media/lmn/86A406A0A406933B2/TNAC_BIDS/'
output_dir = 'derivatives/masks/output_inverse_transform_ROIs'
working_dir = 'derivatives/masks/workingdir_inverse_transform_ROIs' 
input_dir_preproc = 'derivatives/preprocessing/output_preproc'
input_dir_reg = 'derivatives/preprocessing/output_registration'

#location of atlas --> downloaded from alpaca
input_dir_ROIs = 'derivatives/anat_rois_norman-haignere/anatlabels_surf_mni/mni152_te11-te10-te12-pt-pp'

# list of subjects
subject_list = ['sub-03', 'sub-04', 'sub-05', 'sub-06', 'sub-07', 'sub-08', 'sub-09', 'sub-10', 'sub-11', 'sub-12', 'sub-13', 'sub-14'] 


In [ ]:
#### specify workflow-nodes #####

# FreeSurferSource - Data grabber specific for FreeSurfer data
fssource = Node(FreeSurferSource(subjects_dir=fs_dir),
                run_without_submitting=True,
                name='fssource')

# Convert FreeSurfer's MGZ format into NIfTI.gz-format (brain.mgz-anatomical)
convert2niigz = Node(MRIConvert(out_type='niigz'), name='convert2niigz')

# Transform the volumetric ROIs to the target space
inverse_transform_rois = MapNode(ApplyTransforms(args='--float',
                                  input_image_type=3,
                                  interpolation='Linear',
                                  invert_transform_flags=[False],
                                  num_threads=1,
                                  terminal_output='file'),
                  name='inverse_transform_rois', iterfield=['input_image'])
 
# Gunzip - unzip the output ROI-images to use them in further DCM-analysis
gunzip_rois = MapNode(Gunzip(), name="gunzip_rois", iterfield=['in_file'])

# Gunzip - unzip the anatomical reference-image to use it in further DCM-analysis
gunzip_anat = Node(Gunzip(), name="gunzip_anat")

In [ ]:
##### specify input and output stream #####

# Infosource - a function free node to iterate over the list of subject names
infosource = Node(IdentityInterface(fields=['subject_id']),
                  name="infosource")
infosource.iterables = [('subject_id', subject_list)]

templates = {'inverse_transform_composite': opj(input_dir_reg, 'registrationtemp', '{subject_id}', 'transformInverseComposite.h5'),
             'atlas_ROIs': opj(input_dir_ROIs, '*.nii.gz')
            } 

# SelectFiles - to grab the data (alternativ to DataGrabber),
selectfiles = Node(SelectFiles(templates,
                               base_directory=experiment_dir),
                   name="selectfiles")

# Datasink - creates output folder for important outputs
datasink = Node(DataSink(base_directory=experiment_dir,
                         container=output_dir),
                name="datasink")

# Use the following DataSink output substitutions
substitutions = [('_subject_id_', '')]

datasink.inputs.substitutions = substitutions

In [ ]:
##### initiate the workflow and connect nodes #####

# Initiation of the inverse transform ROIs workflow
inverse_transform_ROIs = Workflow(name='inverse_transform_ROIs')
inverse_transform_ROIs.base_dir = opj(experiment_dir, working_dir)

# Connect up ANTS normalization components
inverse_transform_ROIs.connect([(fssource, convert2niigz, [('brain', 'in_file')]),            
                  (convert2niigz, inverse_transform_rois, [('out_file', 'reference_image')]),              
                  (inverse_transform_rois, gunzip_rois, [('output_image', 'in_file')]),
                  (convert2niigz, gunzip_anat, [('out_file', 'in_file')]),              
                  ])

In [ ]:
# Connect SelectFiles and DataSink to the workflow
inverse_transform_ROIs.connect([(infosource, selectfiles, [('subject_id', 'subject_id')]),
                  (infosource, fssource, [('subject_id', 'subject_id')]),         
                  (selectfiles, inverse_transform_rois, [('atlas_ROIs', 'input_image')]),
                  (selectfiles, inverse_transform_rois, [('inverse_transform_composite', 'transforms')]),
                                
                  (convert2niigz, datasink, [('out_file', 'convert2niigz.@anatomical_niigz_transform')]),              
                  (gunzip_rois, datasink, [('out_file', 'inverse_transform_rois.@roi_transform')]),
                  (gunzip_anat, datasink, [('out_file', 'unzipped_anatomical.@unzipped_anatomical')]),              
                  ])

In [ ]:
##### visualize the pipeline #####

# Create a colored output graph
inverse_transform_ROIs.write_graph(graph2use='colored',format='png', simple_form=True)

# Create a detailed output graph
inverse_transform_ROIs.write_graph(graph2use='flat',format='png', simple_form=True)

In [ ]:
# Visualize the simple graph
from IPython.display import Image
Image(filename='/media/lmn/86A406A0A406933B2/TNAC_BIDS/derivatives/masks/workingdir_inverse_transform_ROIs/inverse_transform_ROIs/graph.png')

In [ ]:
# Visualize the detailed graph
from IPython.display import Image
Image(filename='/media/lmn/86A406A0A406933B2/TNAC_BIDS/derivatives/masks/workingdir_inverse_transform_ROIs/inverse_transform_ROIs/graph_detailed.png')

In [ ]:
##### run the workflow using multiple cores #####

inverse_transform_ROIs.run('MultiProc', plugin_args={'n_procs':4})

In [ ]:
!tree /media/lmn/86A406A0A406933B2/TNAC_BIDS/derivatives/masks/output_inverse_transform_ROIs/